In [26]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from src.densenet.densenet import CustomDenseNet
from src.densenet.runtime_transforms import RuntimeTransforms
import os
import glob
from PIL import Image
from tqdm import tqdm
from src.densenet.compute_mAP import compute_mAP_for_ks_numpy
import datetime

In [27]:
os.getcwd()

'C:\\Users\\wlwee\\OneDrive\\Documents\\sunflower-star-photo-reid\\src\\misc'

In [28]:
!cd

C:\Users\wlwee\OneDrive\Documents\sunflower-star-photo-reid\src\misc


In [29]:
path_models_root = os.path.join('..', '..', 'models', 'reid')
path_model_dirs = glob.glob(os.path.join(path_models_root, '*'))
print(f'found {len(path_model_dirs)} models')

found 513 models


In [30]:
path_model_settings = [os.path.join(path_model_dir, 'logs', 'settings.txt') for path_model_dir in path_model_dirs]
print(f'found {len(path_model_settings)} settings files')

found 513 settings files


In [31]:
def get_number_of_epochs(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if "Number of Epochs" in line:
                # Split the line into key and value parts
                key, value = line.split(':')
                # Return the value part as an integer
                return int(value.strip())

def get_number_of_head_layers(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if "Number of Layers" in line:
                # Split the line into key and value parts
                key, value = line.split(':')
                # Return the value part as an integer
                return int(value.strip())

def get_embedding_size(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if "Embedding Size" in line:
                # Split the line into key and value parts
                key, value = line.split(':')
                # Return the value part as an integer
                return int(value.strip())

In [32]:
target_num_epochs = 5
path_model_settings = [path for path in path_model_settings if get_number_of_epochs(path) == target_num_epochs]
print(f'found {len(path_model_settings)} settings files')

found 347 settings files


In [33]:
path_models = [os.path.join(os.path.dirname(os.path.dirname(path)), 'best_densenet121__PT-True.pth') for path in path_model_settings]
print(f'found {len(path_models)} models')

found 347 models


In [34]:
datasets_root = os.path.join('..', '..', 'data')
print(f'{os.path.exists(datasets_root)}')

True


In [35]:
model_datasets_dict = {}
for path_model in path_models:
    dataset = os.path.basename(os.path.dirname(path_model)).split('__')[0]
    print(f'for model {path_model}\n\t dataset is {dataset}')
    path_dataset = os.path.join(datasets_root, dataset)
    print(f'\t dataset path is {path_dataset}')
    model_datasets_dict[path_model] = path_dataset

for model ..\..\models\reid\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__0__densenet121__PT-True\best_densenet121__PT-True.pth
	 dataset is Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
	 dataset path is ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
for model ..\..\models\reid\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__1__densenet121__PT-True\best_densenet121__PT-True.pth
	 dataset is Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
	 dataset path is ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
for model ..\..\models\reid\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-06-02-00-08__0__densenet121__PT-True\best_densenet121__PT-True.pth
	 dataset is Other_species_sea-star-re-id-fu

In [36]:
evaluations_root = os.path.join('..', '..', 'evaluations', 'mAP')
if not os.path.exists(evaluations_root):
    os.makedirs(evaluations_root)


In [37]:
date_txt_now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
path_results_root = os.path.join(evaluations_root, f'mAP_{date_txt_now}')
if not os.path.exists(path_results_root):
    os.makedirs(path_results_root)

path_results_dict = {}

k_values = [1, 5, 10, 15, 20]
n = 0
for path_model, path_dataset in model_datasets_dict.items():
    print(f'{n+1}|{len(model_datasets_dict)} - {os.path.basename(os.path.dirname(path_model))} - {os.path.basename(path_dataset)}')
    path_model_dir = os.path.dirname(path_model)

    path_settings = os.path.join(path_model_dir, 'logs', 'settings.txt')
    number_of_layers = get_number_of_head_layers(path_settings)
    embedding_size = get_embedding_size(path_settings)

    if number_of_layers == None or embedding_size == None:
        print(f'number of layers is: {number_of_layers}. embedding dim is {embedding_size}')
        n += 1
        continue

    # Instantiate the model
    model = CustomDenseNet(version='densenet121', pretrained=False,
                           embedding_dim=embedding_size, num_embed_layers=number_of_layers)  # Set pretrained=False as we'll load our weights

    # Load the trained weights
    model.load_state_dict(torch.load(path_model))
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Check if the model has been loaded correctly
    model.eval()

    # Gallery dataset path
    #path_gallery = r'..\..\data\Pycnopodia_helianthoides_full_event-unaware_training-split_2024-05-03-18-41\train'
    path_gallery = os.path.join(path_dataset, 'train')
    print(f'Gallery dataset path: {path_gallery} {os.path.exists(path_gallery)}')
    ## each subdirectory in the gallery is a identity class
    ## each subdirectory contains images of that identity
    ## the subdirectory name is the identity
    ## the images are in png or jpg format
    ## we need to make a code to num dictionary
    ## we need to load each image and pass it through the model to get the embeddings
    ## we need to store the embeddings in a list of length equal to the number of images
    ## we need to store the identity in a list of length equal to the number of images

    # Query dataset path
    #path_query = r'..\..\data\Pycnopodia_helianthoides_full_event-unaware_training-split_2024-05-03-18-41\test'
    path_query = os.path.join(path_dataset, 'test')
    print(f'Query dataset path: {path_query} {os.path.exists(path_query)}')
    ## each subdirectory in the gallery is a identity class
    ## each subdirectory contains images of that identity
    ## the subdirectory name is the identity
    ## the images are in png or jpg format
    ## we need to make a code to num dictionary
    ## these need to be the same as the gallery dataset
    ## we need to load each image and pass it through the model to get the embeddings
    ## we need to store the embeddings in a list of length equal to the number of images
    ## we need to store the identity in a list of length equal to the number of images

    def get_id (path):
        return os.path.basename(os.path.dirname(path))

    path_gallery_images = glob.glob(os.path.join(path_gallery, '*/*.png'))
    path_gallery_images += glob.glob(os.path.join(path_gallery, '*/*.jpg'))
    gallery_ids = [get_id(path) for path in path_gallery_images]

    path_query_images = glob.glob(os.path.join(path_query, '*/*.png'))
    path_query_images += glob.glob(os.path.join(path_query, '*/*.jpg'))
    query_ids = [get_id(path) for path in path_query_images]

    ids_dict = {id: i for i, id in enumerate(sorted(list(set(gallery_ids + query_ids))))}
    num_dict = {i: id for i, id in enumerate(sorted(list(set(gallery_ids + query_ids))))}

    gallery_ids = [ids_dict[id] for id in gallery_ids]
    query_ids = [ids_dict[id] for id in query_ids]

    print(f'Number of identities: {len(ids_dict)}')
    print(f'Number of gallery images: {len(path_gallery_images)}')
    print(f'Number of query images: {len(path_query_images)}')

    # Initialize the runtime transforms
    runtime_transforms = RuntimeTransforms(h=256, w=256, resize_ratio=0.5)

    gallery_embeddings = []
    pbar = tqdm(total=len(path_gallery_images), position=0, leave=True)
    for path in path_gallery_images:
        img = Image.open(path).convert("RGB")
        transformed_images = runtime_transforms.apply_transforms(img) ## returns a list of transformed images

        # Embed all the transformed images at once as a batch
        batch = torch.stack([transformed_image for transformed_image in transformed_images]).to(device)
        with torch.no_grad():
            embeddings = model(batch).cpu().numpy()
        # Average the embeddings
        averaged_embedding = np.mean(embeddings, axis=0)
        gallery_embeddings.append(averaged_embedding)
        pbar.update(1)
    pbar.close()

    query_embeddings = []
    pbar = tqdm(total=len(path_query_images), position=0, leave=True)
    for path in path_query_images:
        img = Image.open(path).convert("RGB")
        transformed_images = runtime_transforms.apply_transforms(img)  # Returns a list of transformed images

        # Embed all the transformed images at once as a batch
        batch = torch.stack([transformed_image for transformed_image in transformed_images]).to(device)
        with torch.no_grad():
            embeddings = model(batch).cpu().numpy()

        # Average the embeddings
        averaged_embedding = np.mean(embeddings, axis=0)
        query_embeddings.append(averaged_embedding)
        pbar.update(1)
    pbar.close()

    # Convert to numpy arrays
    gallery_embeddings = np.array(gallery_embeddings)
    query_embeddings = np.array(query_embeddings)
    gallery_ids = np.array(gallery_ids)
    query_ids = np.array(query_ids)

    # Compute mAP using the computed embeddings
    mAPs_computed = compute_mAP_for_ks_numpy(query_embeddings, gallery_embeddings, query_ids, gallery_ids, k_values)

    print("\nComputed mAP:")
    mAP_dict = {k: mAPs_computed[k] for k in k_values}
    for k in k_values:
        print(f"mAP@{k}: {mAP_dict[k]:.4f}")
    # save mAP dict
    path_npy = os.path.join(path_model_dir, 'logs', 'mAP_dict.npy')
    np.save(path_npy, mAP_dict)

    ## define a dataframe to store the embeddings
    ## columns will be ['image_path', 'identity', 'gallery/query', 'embedding']
    ## save the dataframe as a pickle file
    dataframe = pd.DataFrame(columns=['image_path', 'identity', 'gallery/query', 'embedding'])
    for i, path in enumerate(path_gallery_images):
        dataframe.loc[i] = [path, gallery_ids[i], 'gallery', gallery_embeddings[i].tolist()]
    for i, path in enumerate(path_query_images):
        dataframe.loc[len(path_gallery_images) + i] = [path, query_ids[i], 'query', query_embeddings[i].tolist()]
    path_embeddings = os.path.join(path_model_dir, 'logs', 'mAP_embeddings.pkl')
    dataframe.to_pickle(path_embeddings)

    path_results_dict[path_model] = [path_model, path_dataset, mAP_dict, path_embeddings]

    n += 1

## save the results dict
path_results_dict_pkl = os.path.join(path_results_root, 'mAP_results_dict.pkl')
pd.to_pickle(path_results_dict, path_results_dict_pkl)


1|347 - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__0__densenet121__PT-True - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
Gallery dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\train True
Query dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\test True
Number of identities: 95
Number of gallery images: 1504
Number of query images: 430


100%|██████████| 430/430 [00:14<00:00, 29.89it/s]



Computed mAP:
mAP@1: 0.8000
mAP@5: 0.9419
mAP@10: 0.9721
mAP@15: 0.9814
mAP@20: 0.9837
2|347 - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__1__densenet121__PT-True - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
Gallery dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\train True
Query dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\test True
Number of identities: 95
Number of gallery images: 1504
Number of query images: 430


100%|██████████| 430/430 [00:21<00:00, 19.85it/s]



Computed mAP:
mAP@1: 0.8093
mAP@5: 0.9395
mAP@10: 0.9651
mAP@15: 0.9814
mAP@20: 0.9907
3|347 - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-06-02-00-08__0__densenet121__PT-True - Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13
Gallery dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\train True
Query dataset path: ..\..\data\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13\test True
Number of identities: 95
Number of gallery images: 1504
Number of query images: 430


...


Computed mAP:
mAP@1: 0.7887
mAP@5: 0.9202
mAP@10: 0.9531
mAP@15: 0.9671
mAP@20: 0.9718
346|347 - Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40__2024-05-09-00-09-37__0__densenet121__PT-True - Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40
Gallery dataset path: ..\..\data\Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40\train True
Query dataset path: ..\..\data\Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40\test True
Number of identities: 95
Number of gallery images: 1508
Number of query images: 426


100%|██████████| 426/426 [00:13<00:00, 30.75it/s]



Computed mAP:
mAP@1: 0.7793
mAP@5: 0.9319
mAP@10: 0.9648
mAP@15: 0.9718
mAP@20: 0.9742
347|347 - Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40__2024-05-09-00-19-46__1__densenet121__PT-True - Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40
Gallery dataset path: ..\..\data\Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40\train True
Query dataset path: ..\..\data\Unmasked_Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-05-15-40\test True
Number of identities: 95
Number of gallery images: 1508
Number of query images: 426


100%|██████████| 426/426 [00:14<00:00, 30.19it/s]



Computed mAP:
mAP@1: 0.8451
mAP@5: 0.9507
mAP@10: 0.9789
mAP@15: 0.9836
mAP@20: 0.9906
